In [1]:
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath

In [2]:
pip install --upgrade scipy==1.4.1

Note: you may need to restart the kernel to use updated packages.


In [45]:
def powerlaw(r, s0, b):  
    # r0 no es variable aleatoria
    r0 = 300
    return s0*(r/r0)**(-b)

def modelo(x, y0, m):
    return y0 + m * x

def funcion_costo(theta, xdata, ydata, ysigma):
        y0 = theta[0]
        m = theta[1]
        costo = np.zeros_like(y0)
        for (x1, y1, sigma1) in zip(xdata, ydata, ysigma):
            residuo = y1 - modelo(x1, y0, m)
            z = residuo / sigma1
            costo += z*z
        return costo
    
costo = lambda theta: funcion_costo(theta, x, y, sigmay)  

def funcion_costo2(theta, xdata, ydata, ysigma):
        s0 = theta[0]
        b = theta[1]
        costo = np.zeros_like(s0)
        for (r, y1, sigma1) in zip(xdata, ydata, ysigma):
            residuo = y1 - powerlaw(r, s0, b)
            z = residuo / sigma1
            costo += z*z
        return costo
    
costo2 = lambda theta: funcion_costo2(theta, r, y_random, sigma) 

def in_hex(poli):
    path = mpltPath.Path(poli)
    hex = False
    random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    while hex != True:
        if path.contains_points([random_point]) == True:
            hex = True
        else:
            random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    return random_point

def getDistances(xpositions, ypositions, x0=0, y0=0, zenithRad=0, azimuthRad=0):

    dx = xpositions - x0    # detector wrt core positions
    dy = ypositions - y0
    
    groundDistances = np.sqrt(dx*dx+dy*dy)

    # Direction cosines in the plane xy
    dirx =  math.cos(azimuthRad)
    diry =  math.sin(azimuthRad)

    axisProjections = math.sin(zenithRad) * (dx*dirx+dy*diry)
    
    distances = np.sqrt( groundDistances**2 - axisProjections**2 )
    
    return distances

def simZenith(zenithMinDeg, zenithMaxDeg) :

    zenithMinRad = math.radians(zenithMinDeg)
    sin2MinZenith = math.sin(zenithMinRad)**2

    zenithMaxRad = math.radians(zenithMaxDeg)
    sin2MaxZenith = math.sin(zenithMaxRad)**2

    rng = np.random.default_rng()
    u = rng.random() 
    
    sin2Zenith = sin2MinZenith + u * (sin2MaxZenith-sin2MinZenith)
    sinZenith = math.sqrt(sin2Zenith) 

    zenith = math.asin(sinZenith)
    azimut = np.random.uniform(0,2*np.pi)
    
    return (zenith,azimut)

poligono = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0]]
rectangulo = [[-433,-375],[433,-433],[433,375],[-433,375]]
pointsx,pointsy = [-216.5,216.5,433,216.5,-216.5,-433.5,0] , [-375,-375,0,375,375,0,0]
random_point = in_hex(poligono)
centro = (0,0)

In [4]:
x = np.linspace(2,3,11)
y = [2.78, 3.29, 3.29, 3.33, 3.23, 3.69, 3.46, 3.87, 3.62, 3.40, 3.99]
sigmay = [0.3]*len(y)

In [5]:
res = minimize(costo, x0=(0,0))
res

      fun: 4.639989898990976
 hess_inv: array([[ 0.25977272, -0.10227272],
       [-0.10227272,  0.04090909]])
      jac: array([ 5.96046448e-08, -1.78813934e-07])
  message: 'Optimization terminated successfully.'
     nfev: 28
      nit: 5
     njev: 7
   status: 0
  success: True
        x: array([1.45227346, 0.79909061])

In [6]:
print("Parametros ajustados: ", res.x)
cova = res.hess_inv
print("Covarianza estimada: ", 2*cova)  
par_error = np.sqrt(np.diagonal(2*cova))
print("Errores de los parámetros", par_error)

Parametros ajustados:  [1.45227346 0.79909061]
Covarianza estimada:  [[ 0.51954543 -0.20454545]
 [-0.20454545  0.08181818]]
Errores de los parámetros [0.720795   0.28603877]


In [58]:
#Defino parametros de LDF 
r0 = 300 #metros 
s0 = 35 #VEM
angulo = 40
b = 2.1    
#Genero el punto random
random_point = in_hex(poligono)
#calculo las distancias entre el evento y los 7 detectores.
distancias = getDistances(np.asarray(pointsx),np.asarray(pointsy),random_point[0],random_point[1],simZenith(-angulo,angulo)[0],simZenith(-angulo,angulo)[1])
r = np.asarray(distancias)


y1 = powerlaw(r,s0,b)
sigma = np.sqrt(y1)

#Hago el ajuste y defino la semilla inicial
y_random = np.random.normal(y1,sigma,size = len(y1))  #signal    
params=[35, 1]
popt, pcov = curve_fit(powerlaw, distancias, y_random, p0=params)
print(f'curve_fit results',popt,np.sqrt(np.diag(pcov)))
print(pcov)

curve_fit results [40.13197817  1.96074644] [2.35368646 0.0782229 ]
[[ 5.53983994 -0.17010585]
 [-0.17010585  0.00611882]]


In [59]:
res = minimize(costo2, x0=(300,2.1))
print('Minimize results')
print("Parametros ajustados: ", res.x)
cova = res.hess_inv
print("Covarianza estimada: ", 2*cova)  
par_error = np.sqrt(np.diagonal(2*cova))
print("Errores de los parámetros", par_error)

Minimize results
Parametros ajustados:  [38.91169847  2.0151886 ]
Covarianza estimada:  [[ 6.04044114 -0.14271768]
 [-0.14271768  0.00882693]]
Errores de los parámetros [2.45773089 0.09395174]
